In [1]:
import torch

# Model
# pip install -U ultralytics
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


Using cache found in C:\Users\tinke/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-6 Python-3.11.3 torch-2.1.0.dev20230604+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)

100%|██████████| 14.1M/14.1M [00:01<00:00, 8.44MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [4]:

# Image
im = 'myapple_1.jpg'

# Inference
results = model(im)

results.pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,67.464676,84.615707,549.757324,559.911804,0.727682,47,apple


In [6]:
# Results
#crops = results.crop(save=True) 
crops = results.crop(save=True, save_dir='apple_cropped/')  # specify save dir

Saved 1 image to apple_cropped
Saved results to apple_cropped

